读取文件

In [ ]:
# from transformers import pipeline
# summarizer = pipeline("summarization")

In [1]:
from bs4 import BeautifulSoup

In [2]:
blacklist = [   '[document]',   'noscript', 'header',   'html', 'meta', 'head','input', 'script',   ]

In [3]:
import ebooklib
from ebooklib import epub
def epub2thtml(epub_path):
    book = epub.read_epub(epub_path)
    chapters = []
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            chapters.append(item.get_content())
    return chapters

In [4]:
def chap2text(chap):
    output = ''
    soup = BeautifulSoup(chap, 'html.parser')
    text = soup.find_all(text=True)
    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)
    return output

In [5]:
def thtml2ttext(thtml):
    Output = []
    for html in thtml:
        text =  chap2text(html)
        Output.append(text)
    return Output

In [6]:
    def epub2text(epub_path):
        chapters = epub2thtml(epub_path)
        ttext = thtml2ttext(chapters)
        return ttext

In [7]:
out=epub2text('./Murder_on_the_Orient_Express.epub')

In [8]:
len(out)

47

In [9]:
part_one = out[5:13]
part_two = out[14:29] 
part_three = out[30:39]

In [10]:
# fold_path = "./raw_text/"
# def save_chapter_by_part(part, chapters):
#     for i in range(len(chapters)):
#         file_name = "Part{0}_Chapter{1}".format(part, i+1)
#         with open(fold_path+file_name, "w") as f:
#             f.write(chapters[i])

In [11]:
# save_chapter_by_part(1,part_one)
# save_chapter_by_part(2,part_two)
# save_chapter_by_part(3,part_three)

### 预处理

#### (1) 大写转小写, 删除掉文本中的对话内容
---
这样做的原因是对话中往往含有很多法语，并且会有很多人名干扰词汇

In [12]:
example_text = part_one[4]

In [14]:
# example_text = example_text.lower()

In [15]:
len(example_text)

17244

In [16]:
import re
def exclude_dialog(text):
    print("all text length: ", len(text))
    code_pattern = re.compile(r'“.*?”', flags=re.DOTALL)
    res = code_pattern.sub(" ", text)
    print("without dialog length: ", len(res))
    return res

In [17]:
res = exclude_dialog(example_text) # 删除对话

all text length:  17244
without dialog length:  8024


#### (2) 处理缩写单词，去除标点符号

In [18]:
import nltk
import re
from nltk.tokenize import ToktokTokenizer
def clean_text(text):
#     text = text.lower()
    text = re.sub(r'\b[A-Z][a-zA-Z]*\b', " ", text)    # 删除大写单词
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text)
    text = re.sub(r"\'\xa0", " ", text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text


In [19]:
from nltk.tokenize import word_tokenize, wordpunct_tokenize

In [20]:
punct = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-—“”’'

In [21]:
def strip_list_noempty(mylist):
    newlist = [item.strip() if hasattr(item, 'strip') else item for item in mylist]
    return [item for item in newlist if item != '']

def clean_punct(text, type = "str"): 
#     words = token.tokenize(text)
    words = word_tokenize(text)
    punctuation_filtered = []
    regex = re.compile('[%s]' % re.escape(punct))
    for w in words:
        punctuation_filtered.append(regex.sub('', w))
    filtered_list = strip_list_noempty(punctuation_filtered)
    if type == "str":
        return ' '.join(map(str, filtered_list))
    else:
        return filtered_list

In [22]:
example_text = clean_text(example_text)   # 处理缩写、处理残缺单词

In [23]:
example_words_list = clean_punct(example_text, 'list')      # 清除标点， 保存成list

In [24]:
example_text = clean_punct(example_text)                    # 清除标点， 保存成str

In [25]:
# 在这一步进行NER，帮助筛选数据
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
doc = nlp(example_text)
NE = set(word for X in doc.ents for word in X.text.split())

In [26]:
len(example_words_list)

2578

In [27]:
from collections import Counter
counter_before = Counter(example_words_list)

In [28]:
len(counter_before)

864

### (3) 词型还原

In [29]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

In [30]:
lemma=WordNetLemmatizer()
def lemitizeWords(text, type="str"):
    words=word_tokenize(text)
    listLemma=[]
    for w in words:
        x=lemma.lemmatize(w, pos="v")
        listLemma.append(x)
    if type == "str":
        return ' '.join(map(str, listLemma))
    else:
        return listLemma

In [31]:
lemma_result_list = lemitizeWords(example_text, type = "list")

In [32]:
example_text = lemitizeWords(example_text)

In [33]:
counter_lemma = Counter(lemma_result_list)

In [34]:
len(counter_lemma)

788

### (4) 去除停顿词

In [35]:
stop_words = set(stopwords.words("english"))
def stopWordsRemove(text, type = "str"):
    stop_words = set(stopwords.words("english"))
    words=word_tokenize(text)
    filtered = [w for w in words if not w in stop_words]
    if type == "str":
        return ' '.join(map(str, filtered))
    else:
        return filtered

In [36]:
remove_stop_result_list = stopWordsRemove(example_text, list)

In [37]:
example_text = stopWordsRemove(example_text)

In [38]:
counter_stop_list = Counter(remove_stop_result_list)

In [39]:
len(counter_stop_list)

689

In [40]:
word_dic = dict(counter_stop_list)      # 删除停顿词后的单词

### (5) 基于词频过滤，得到最终的词汇表

In [41]:
from wordfreq import zipf_frequency

In [43]:
word_freq = {}
for w, c in word_dic.items():
    word_freq[w] = zipf_frequency(w, 'en')

In [44]:
freq_list = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)

根据对单词的认识程度，发现需要预先学习的单词的log词频应该在4.15一下 (词频为0的单词除外)

In [45]:
my_level = 4.15

In [51]:
word_dic

{'e': 1,
 'find': 2,
 'difficult': 3,
 'go': 7,
 'sleep': 3,
 'one': 13,
 'thing': 3,
 'miss': 2,
 'motion': 1,
 'train': 21,
 'station': 2,
 'outside': 3,
 'curiously': 1,
 'quiet': 2,
 'contrast': 1,
 'noise': 2,
 'seem': 6,
 'unusually': 1,
 'loud': 1,
 'could': 1,
 'hear': 8,
 'move': 2,
 'next': 3,
 'doora': 1,
 'click': 2,
 'pull': 1,
 'washbasin': 1,
 'sound': 3,
 'tap': 3,
 'run': 5,
 'splash': 1,
 'another': 1,
 'basin': 1,
 'shut': 2,
 'pass': 3,
 'corridor': 4,
 'shuffle': 1,
 'footsteps': 2,
 'someone': 2,
 'bedroom': 1,
 'slippers': 1,
 'lay': 3,
 'awake': 2,
 'star': 1,
 'ceiling': 1,
 'silent': 1,
 'throat': 1,
 'felt': 2,
 'dry': 1,
 'forget': 2,
 'ask': 11,
 'usual': 1,
 'bottle': 1,
 'mineral': 2,
 'water': 5,
 'look': 15,
 'watch': 2,
 'quarter': 2,
 'past': 4,
 'would': 8,
 'ring': 1,
 'conductor': 13,
 'finger': 4,
 'bell': 3,
 'pause': 1,
 'stillness': 1,
 'ting': 1,
 'man': 21,
 'answer': 3,
 'every': 1,
 '…ting…ting…': 1,
 'get': 6,
 'impatient': 2,
 '…': 1,
 'k

In [52]:
valuable_word = []
for w, f in word_freq.items():
    if 0 < f < my_level and w not in NE and word_dic[w]>1:
        valuable_word.append(w)

In [53]:
len(valuable_word)

22

In [65]:
import json
import sys
from urllib.parse import urlparse, quote, urlencode, unquote
from urllib.request import urlopen
import re
 
def fetch(query_str):
    query = {'q': "".join(query_str)}   # list --> str: "".join(list)
    url = 'https://fanyi.youdao.com/openapi.do?keyfrom=11pegasus11&key=273646050&type=data&doctype=json&version=1.1&' + urlencode(query)
    response = urlopen(url, timeout=3)
    html = response.read().decode('utf-8')
    return html

def prase(html):
    prased = json.loads(html)
    if 'basic' in prased and 'explains'in prased['basic']:
        explain_list = prased['basic']['explains']
        for i in range(len(explain_list)):
            if "人名" in explain_list[i] or "地名" in explain_list[i]:    # 将人名地名的翻译删掉
                explain_list[i] = ""
        return " ".join(prased['basic']['explains'])
    else:
        return "Unknown"




In [66]:
# 翻译单词
import time
vocabulary = []
for w in valuable_word:
    vocabulary.append([w, prase(fetch(w))])

In [67]:
for item in vocabulary:
    print("{}\t\t{}".format(item[0], item[1]))

corridor		n. 走廊
footsteps		n. 脚步（footstep的复数形式）；步距
mineral		n. 矿物；（英）矿泉水；无机物；苏打水（常用复数表示） adj. 矿物的；矿质的
conductor		n. 导体；售票员；领导者；管理人
impatient		adj. 焦躁的；不耐心的
wipe		n. 擦拭；用力打 vi. 擦；打 vt. 擦；消除；涂上
insist		vi. 坚持，强调 vt. 坚持，强调
compartment		n. [建] 隔间；区划；卧车上的小客房 vt. 分隔；划分
snowdrift		n. 随风飘飞的雪；被风刮在一起的雪堆
standstill		n. 停顿；停止
glance		v. 瞥闪，瞥见，扫视，匆匆一看；浏览；斜击某物后斜弹开来；反光；轻擦（球）；（板球）斜击  n. 一瞥；一滑；闪光；（板球）斜击；辉金属
valet		n. 贴身男仆；用车的人；伺候客人停车 vt. 为...管理衣物；替...洗熨衣服 vi. 清洗汽车；服侍 
polite		adj. 有礼貌的，客气的；文雅的；上流的；优雅的
din		 n. 喧嚣声，嘈杂声；宗教法律，犹太法律；（尤指伊斯兰教）宗教信仰 v. 再三叮嘱，反复教导；发出喧闹声
bury		vt. 埋葬；隐藏 
comprehend		vt. 理解；包含；由…组成
murderer		n. 凶手；谋杀犯
thoughtfully		adv. 沉思地；体贴地，亲切地
stab		n. 刺；戳；尝试；突发的一阵 vt. 刺；刺伤；戳；刺穿；直入 vi. 刺；刺伤；戳；刺痛 
assure		vt. 保证；担保；使确信；弄清楚
nod		v. 点头；点头表示；打盹；摆动；（英足）顶球 n. 点头；点头表示；打盹；摆动；同意，让步
inconvenience		n. 不便；麻烦 vt. 麻烦；打扰


In [ ]:
class ReadHelper:
    def __init__(book_path, language_level):
        this.book_path = book_path
        this.language_level = language_level
        
    def
        